# Implementing the gradient descent algorithm
---
Let's first use `sklearn` to predict housing prices based on the sample dataset. Then let's implement gradient descent and see if we come up with the same predicted housing prices.

In [2]:
import numpy as np
import pandas as pd
import random
import math
import matplotlib.pyplot as plt
%matplotlib inline
from scipy.spatial import distance
import sklearn
from scipy import stats
from sklearn.datasets.samples_generator import make_regression 

living_area = np.array([2104, 1600, 2400, 1416, 3000])
num_bedrooms = np.array([3, 3, 3, 2, 4])
price = np.array([400, 330, 369, 232, 540]) # we want to predict the price
x = np.concatenate((living_area, num_bedrooms), axis = 0)
x = np.concatenate((x, np.ones(len(num_bedrooms))), axis = 0) # add in 1s so that we can use theta_0 when computing dot product
x = x.reshape(3, 5).T

from sklearn import linear_model
clf = linear_model.LinearRegression()
clf.fit(x, price)

print "coefficients of the hypothesis: ", clf.coef_

for i, sample in enumerate(x):
    predicted_price = 0
    predicted = np.dot(sample, clf.coef_)
    print "predicted: $%f, actual: $%f" % (predicted, price[i])

coefficients of the hypothesis:  [  6.38433756e-02   1.03436047e+02   0.00000000e+00]
predicted: $444.634602, actual: $400.000000
predicted: $412.457541, actual: $330.000000
predicted: $463.532241, actual: $369.000000
predicted: $297.274313, actual: $232.000000
predicted: $605.274313, actual: $540.000000


## Batch Gradient Descent
Gradient descent can be implemented in two main ways: batch gradient descent or stochastic gradient descent. Below is an implementation of batch gradient descent.

In [15]:
# X = training dataset, or input data that'll allow us to predict the target dataset (below)
# Y = target dataset, or variable we're trying to predict
# H = hypothesis, the function that'll predict Y

def partial_of_cost_wrt_j(Y, X, theta, H, i, j):
    return (Y[i] - H(X[i], theta)) * X[i][j]
        
def do_gradient_descent(theta, X, Y, H, alpha):
    new_theta = [v for v in theta]
    for j in range(len(theta)): # for each feature in theta
        delta = sum([partial_of_cost_wrt_j(Y, X, theta, H, i, j) for (i, _) in enumerate(Y)])
        new_theta[j] += alpha * delta
    return new_theta

# least squares cost function
def cost(X, Y, H, theta):
    return 0.5 * sum([ (H(X[i], theta) - Y[i])**2 for (i, _) in enumerate(Y) ])
    
def fit(X, Y, learning_rate):
    def hypothesis(X_i, theta):
        return np.dot(X_i, theta)
    
    theta = np.ones(x.shape[1]) # initialize parameters to all be 1's
#     learning_rate = 0.00000001  # this learning rate was empirically determined
    c = cost(X, Y, hypothesis, theta)
    
    num_iterations = 0
    while c > 100:
        num_iterations += 1
#         print "cost: %f, theta: %s" % (cost(X, Y, hypothesis, theta), theta)
        theta = do_gradient_descent(theta, X, Y, hypothesis, learning_rate)
        if abs(cost(X, Y, hypothesis, theta) - c) < 0.001:
            break
        c = cost(X, Y, hypothesis, theta)
    print "with learning_rate = %s, BGD converged after %d iterations" % (learning_rate, num_iterations)
    return theta

coefficients = fit(x, price, 0.000000001)
coefficients = fit(x, price, 0.00000001)

print
print "coefficients of the hypothesis: %s" % coefficients
for i, sample in enumerate(x):
    predicted = np.dot(sample, coefficients)
    print "predicted: $%f, actual: $%f" % (predicted, price[i])

with learning_rate = 1e-09, BGD converged after 412 iterations
with learning_rate = 1e-08, BGD converged after 42 iterations

coefficients of the hypothesis: [0.17515718101812017, 0.99887165461797678, 0.99963822144941095]
predicted: $372.526962, actual: $400.000000
predicted: $284.247743, actual: $330.000000
predicted: $424.373488, actual: $369.000000
predicted: $251.019950, actual: $232.000000
predicted: $530.466668, actual: $540.000000


The `fit` function above uses **batch gradient descent (BGD)** to find the parameters **theta** that result in as low a **cost** as possible. `fit` stops when gradient descent **converges**. Convergence is determined by the programmer. In this case, I printed out the cost after every iteration of BGD, and I determined that it was not changing too much relatively speaking. One value that you need to pay attention to is the **learning rate**. The BGD algorithm **may never converge if the learning rate is too high**. Set the learning rate to be 10x higher, and see what happens to the cost after every iteration of BGD. You'll notice that the cost will increase after every iteration of BGD. This means BGD is taking big strides that are making the algorithm miss a point that'll result in a lower cost. If you set the learning rate to be 10x lower, you'll notice BGD will require more iterations to converge (412 compared to 42).

The reason the learning rate needs to be so low is the features of training set vary greatly in terms of value. The first feature is **living area**, which is typically in the thousands. The second features is **number of bedrooms**, which is usually a single digit number (and very rarely in the double digits, unless you're living in a giant mansion). Furthermore, the target values, the prices of the homes, are in the hundreds. With the initial values of theta all set to 1's, the hypothesis will initially evaluate to numbers nearing the thousands because living area is by far the largest feature value. Then, if you subtract the price (which are in the hundreds), you'll still end up with an error (hypothesis - target) in the thousands. As a result, each value in the vector theta will change greatly and wildly if the learning rate is too high. So we must set the learning rate to be low to ensure that theta doesn't change too greatly. If it changes too greatly, intuitively, you can visualize BGD taking huge strides and completely missing values for which theta will bring down the cost.

What if we wanted higher learning rates, that is, in the 0.001 to 0.1 range? As explained above, the variance among the ranges of feature values (thousands for living area vs single digit numbers for number of living rooms) is the source of the problem. What if we could assign a score to the living area of a training sample and a score the number of living rooms of that same training sample such that these scores are on the same scale? Fortunately, we can do that with a technique called **feature scaling**. One way to scale a feature would be to divide all feature values by the maximum value for that specific feature among the training set. For example, the maximum living area in our training set is 3000, so we can divide all living areas by 3000. Below, I've calculated the z-score (from statistics) of each feature value. The z-score is used to compare values in statistics, so that is why I'm using it to scale the features. The only value to not scale are the intercept terms (of 1s).

In [14]:
def scale(X):
    all_features = []
    for i,features in enumerate(X.T[:-1]):
        df = pd.DataFrame(features)
        mean = df.mean().get(0)
        sigma = df.std().get(0)
        scaled_features = [(f - mean) / sigma for f in features]
        all_features.append(scaled_features)
    all_features.append(X.T[-1])
    return np.array(all_features).T

coefficients = fit(scale(x), price, 0.01)
coefficients = fit(scale(x), price, 0.1)
coefficients = fit(scale(x), price, 0.2)

print
print "coefficients of the hypothesis: %s" % coefficients
for i, sample in enumerate(scale(x)):
    predicted = np.dot(sample, coefficients)
    print "predicted: $%f, actual: $%f" % (predicted, price[i])


with learning_rate = 0.01, BGD converged after 742 iterations
with learning_rate = 0.1, BGD converged after 97 iterations
with learning_rate = 0.2, BGD converged after 52 iterations

coefficients of the hypothesis: [40.689903824446866, 73.052403719768009, 374.19999999999999]
predicted: $374.200000, actual: $400.000000
predicted: $341.953257, actual: $330.000000
predicted: $393.138563, actual: $369.000000
predicted: $226.868937, actual: $232.000000
predicted: $534.839243, actual: $540.000000


In [21]:
# works
def hypothesis(X, theta):
    return np.dot(X, theta)

def partial_of_cost_wrt_j(Y, X, theta, H, i, j):
    print "X[i]:", X[i]
    print "i:", i
    print "j:", j
    print "Y[i]:", Y[i]
    print "theta:", theta
    print "H:", H(X[i], theta)
    return (Y[i] - H(X[i], theta)) * X[i][j]

print "partial: ", partial_of_cost_wrt_j(price, x, np.array([1,1,1]), hypothesis, 0, 0)
# works
def sum_of_errors(X, Y, H, theta):
    return Y - np.apply_along_axis(H, 1, X, theta)

def direction(X, Y, H, feature_index, theta):
    return np.dot(sum_of_errors(X, Y, H, theta), X.T[feature_index])

def step(X, Y, H, theta, learning_rate):
    return np.array([theta_j + learning_rate * direction(X, Y, H, feature_index, theta) \
                     for (feature_index, theta_j) in enumerate(theta)])

def cost(X, Y, H, theta):
    return 0.5 * np.sum(np.square(-1 * sum_of_errors(X, Y, H, theta)))

# print "training_set:", x
# theta = np.random.random(x.shape[1])
theta = np.array([1,1,1])
print "theta:", theta
print "price:", price

print "sum_of_errors:", sum_of_errors(x, price, hypothesis, theta)

# http://www.bogotobogo.com/python/python_numpy_batch_gradient_descent_algorithm.php

 partial:  X[i]: [  2.10400000e+03   3.00000000e+00   1.00000000e+00]
i: 0
j: 0
Y[i]: 400
theta: [1 1 1]
H: 2108.0
-3593632.0
theta: [1 1 1]
price: [400 330 369 232 540]
sum_of_errors: [-1708. -1274. -2035. -1187. -2465.]


In [10]:
def train(X, Y, theta):
    iteration = 0
    while True:
        print iteration
        current_cost = cost(X, Y, hypothesis, theta)
        new_theta = step(X, Y, hypothesis, theta, 0.001)
        new_cost = cost(X, Y, hypothesis, new_theta)
        print "current:", current_cost
        print "new:", new_cost

        if abs(new_cost - current_cost) < 0.01 or iteration > 5000:
            break
        theta = new_theta
        iteration += 1
    print theta
    
def gradient_descent(alpha, x, y, ep=0.0001, max_iter=10000):
    converged = False
    iter = 0
    m = x.shape[0] # number of samples

    # initial theta
    t0 = np.random.random(x.shape[1])
    t1 = np.random.random(x.shape[1])

    # total error, J(theta)
    J = sum([(t0 + t1*x[i] - y[i])**2 for i in range(m)])

    # Iterate Loop
    while not converged:
        # for each training sample, compute the gradient (d/d_theta j(theta))
        grad0 = 1.0/m * sum([(t0 + t1*x[i] - y[i]) for i in range(m)]) 
        grad1 = 1.0/m * sum([(t0 + t1*x[i] - y[i])*x[i] for i in range(m)])

        # update the theta_temp
        temp0 = t0 - alpha * grad0
        temp1 = t1 - alpha * grad1
    
        # update theta
        t0 = temp0
        t1 = temp1

        # mean squared error
        e = sum( [ (t0 + t1*x[i] - y[i])**2 for i in range(m)] ) 

        if abs(J-e) <= ep:
            print 'Converged, iterations: ', iter, '!!!'
            converged = True
    
        J = e   # update error 
        iter += 1  # update iter
    
        if iter == max_iter:
            print 'Max interactions exceeded!'
            converged = True

    return t0,t1
    
print "Tests start"
x, y = make_regression(n_samples=100, n_features=1, n_informative=1, random_state=0, noise=35) 
print 'x.shape = %s y.shape = %s' %(x.shape, y.shape)
alpha = 0.01 # learning rate
ep = 0.01 # convergence criteria

# call gredient decent, and get intercept(=theta0) and slope(=theta1)
theta0, theta1 = gradient_descent(alpha, x, y, ep, max_iter=1000)
print ('theta0 = %s theta1 = %s') %(theta0, theta1)
from sklearn import linear_model
clf = linear_model.LinearRegression()
clf.fit(x, y)
print clf.coef_

# x, y = make_regression(n_samples=100, n_features=1, n_informative=1, random_state=0, noise=35)
# print train(x[:,0], y, theta)
# print (sum_of_errors(X, Y, hypothesis) == [10, 10]).all()
# print direction(X, Y, hypothesis, 2) == 80
# print (step(X, Y, hypothesis, np.ones(X.shape[1]), 0.01) == [1.4, 1.6, 1.8, 2.0]).all()
print "Tests end"

Tests start
x.shape = (100, 1) y.shape = (100,)
Converged, iterations:  641 !!!
theta0 = [-2.81942064] theta1 = [ 43.13891807]
[ 43.20424388]
Tests end


In [7]:
# print train(x, price, theta)
# print (sum_of_errors(X, Y, hypothesis) == [10, 10]).all()
# print direction(X, Y, hypothesis, 2) == 80
# print (step(X, Y, hypothesis, np.ones(X.shape[1]), 0.01) == [1.4, 1.6, 1.8, 2.0]).all()

# import matplotlib.pyplot as plt
# from mpl_toolkits.mplot3d import Axes3D
# fig = plt.figure()
# ax = fig.add_subplot(111, projection='3d')
# ax.scatter(living_area, num_bedrooms, price)
# ax.set_xlabel('sq ft')
# ax.set_ylabel('# bedrooms')
# ax.set_zlabel('price ($)')
# # x = y = np.arange(-3.0, 3.0, 0.05)
# # X, Y = np.meshgrid(x, y)
# # zs = np.array([x+y for x,y in zip(np.ravel(X), np.ravel(Y))])
# # Z = zs.reshape(X.shape)
# # ax.plot_surface(X, Y, Z)
# for ii in xrange(0,360,1):
#     ax.view_init(elev=10., azim=ii)

coefficients of the hypothesis:  [  6.38433756e-02   1.03436047e+02   0.00000000e+00]
predicted: $444, actual: $400
predicted: $412, actual: $330
predicted: $463, actual: $369
predicted: $297, actual: $232
predicted: $605, actual: $540
